### Movies


In [2]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import sklearn.cluster as clust
from scipy.spatial.distance import cdist
import os

In [14]:
# import data
df_all = pd.read_excel('movies.xls',sheet_name = None)
df_1950s = df_all['1900s']
df_2000s = df_all['2000s']
df_2010s = df_all['2010s']

In [19]:
df_2010s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1604 entries, 0 to 1603
Data columns (total 25 columns):
Title                          1604 non-null object
Year                           1497 non-null float64
Genres                         1604 non-null object
Language                       1599 non-null object
Country                        1600 non-null object
Content Rating                 1411 non-null object
Duration                       1594 non-null float64
Aspect Ratio                   1373 non-null float64
Budget                         1350 non-null float64
Gross Earnings                 1145 non-null float64
Director                       1500 non-null object
Actor 1                        1600 non-null object
Actor 2                        1594 non-null object
Actor 3                        1592 non-null object
Facebook Likes - Director      1500 non-null float64
Facebook Likes - Actor 1       1600 non-null float64
Facebook Likes - Actor 2       1594 non-null float64
F

In [20]:
df_2010s.describe()

,Year,Duration,Aspect Ratio,Budget,Gross Earnings,Facebook Likes - Director,Facebook Likes - Actor 1,Facebook Likes - Actor 2,Facebook Likes - Actor 3,Facebook Likes - cast Total,Facebook likes - Movie,Facenumber in posters,User Votes,Reviews by Users,Reviews by Crtiics,IMDB Score
count,1497.000000,1594.000000,1373.000000,1.350000e+03,1.145000e+03,1500.000000,1600.000000,1594.000000,1592.000000,1604.000000,1604.000000,1591.000000,1.604000e+03,1588.000000,1571.000000,1604.000000
mean,2012.773547,103.328733,2.619148,4.056324e+07,5.780677e+07,552.748000,7201.725000,2059.250941,843.062186,11036.882170,18269.607855,1.656191,8.063246e+04,210.315491,187.586887,6.337718
std,1.868725,27.429001,2.453727,5.422299e+07,8.050245e+07,2525.020601,14504.056836,4323.170579,2271.266551,18698.435468,29834.279124,2.338754,1.287577e+05,296.486843,165.281572,1.169382
min,2010.000000,7.000000,1.330000,1.400000e+03,1.620000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000e+00,1.000000,1.000000,1.600000
25%,2011.000000,92.000000,1.850000,5.000000e+06,5.283379e+06,6.750000,600.750000,284.000000,127.000000,1399.750000,0.000000,0.000000,4.358500e+03,31.000000,38.000000,5.700000
50%,2013.000000,102.000000,2.350000,2.000000e+07,3.149427e+07,46.000000,1000.000000,599.500000,374.000000,3367.500000,1000.000000,1.000000,3.224250e+04,118.500000,159.000000,6.400000
75%,2014.000000,114.750000,2.350000,5.000000e+07,7.300094e+07,176.000000,12000.000000,969.750000,676.000000,15717.750000,25000.000000,2.000000,9.986700e+04,269.000000,288.000000,7.100000
max,2016.000000,511.000000,16.000000,6.000000e+08,6.521773e+08,23000.000000,260000.000000,27000.000000,23000.000000,283939.000000,349000.000000,31.000000,1.468200e+06,3018.000000,813.000000,9.500000
